In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping

print(tf.__version__)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_data = "train-data.tsv"
test_data = "valid-data.tsv"

In [ ]:
# Load data into DataFrames
train_data = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
test_data = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

# Convert labels to numerical values
train_labels = train_data["label"].map({"ham": 0, "spam": 1})
test_labels = test_data["label"].map({"ham": 0, "spam": 1})

# Extract messages
train_messages = train_data["message"].values
test_messages = test_data["message"].values

In [ ]:
# Tokenize messages
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_messages)

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_messages)
test_sequences = tokenizer.texts_to_sequences(test_messages)

# Pad sequences
max_length = 50
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Get vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary size: {vocab_size}")

In [ ]:
# Build model
model = Sequential([
    Embedding(input_dim=5000, output_dim=32, input_length=max_length),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Explicitly build the model by passing sample input
sample_input = np.ones((1, max_length))
model(sample_input)
model.summary()

In [ ]:
# Train with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_padded,
    train_labels,
    epochs=20,
    validation_data=(test_padded, test_labels),
    batch_size=32,
    callbacks=[early_stop],
    verbose=2
)

In [ ]:
def predict_message(pred_text):
    # Preprocess input text
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')

    # Make prediction
    prediction = model.predict(padded, verbose=0)[0][0]

    # Determine label
    label = "ham" if prediction < 0.5 else "spam"

    return [float(prediction), label]

# Test prediction
print(predict_message("how are you doing today?"))

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
